In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
from torch.utils.tensorboard import SummaryWriter
import random
import collections

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels

In [ ]:
writer = SummaryWriter()

In [ ]:
dataset = CovidDataset(csv_path='./train.csv', transform=transforms.Compose([Rescale(64), ToTensor()]))
print(type(dataset))

In [ ]:
print(len(dataset))

In [ ]:
for i in range(len(dataset)):
    item = dataset.__getitem__(i)
    print("item ", i , " image shape= ", item['image'].shape)

In [ ]:
dset = getActualImgs(dataset, [0,1,2], 1)
print(type(dset))

In [ ]:
for batch_idx, (data, target) in enumerate(dset):
    print(data)

In [4]:

a = np.random.choice(3, 2, replace=False)
b = np.random.choice(3, 2, replace=False)
c = np.random.choice(3, 2, replace=False)
print(a, b, c)
# collections.Counter(a)

[0 1] [1 2] [1 0]


In [7]:
classes_indx = [i for i in range(3)]
print(classes_indx)

[0, 1, 2]


In [134]:
# non iid data test
c_num = 3
num_users = 10
classes, images = c_num, int(300/c_num)
classes_indx = [i for i in range(classes)]
print("classes indx ", classes_indx)
users_dict = {i: np.array([]).astype(int) for i in range(num_users)}
print("users_dict", users_dict)
indeces = np.arange(classes*images)
unsorted_labels = np.random.choice(3, 300)

indeces_unsortedlabels = np.vstack((indeces, unsorted_labels))
# print("indeces_unsortedlabels before shuffle: ", indeces_unsortedlabels)
shuffled_indices = np.random.permutation(len(indeces_unsortedlabels[0]))
indeces_unsortedlabels[0] = indeces_unsortedlabels[0][shuffled_indices]
indeces_unsortedlabels[1] = indeces_unsortedlabels[1][shuffled_indices]
# print("indeces_unsortedlabels after shuffle: ", indeces_unsortedlabels)
indeces_labels = indeces_unsortedlabels[:, indeces_unsortedlabels[1, :].argsort()]
indeces = indeces_labels[0, :]
indeces_labels.astype(int)
indeces.astype(int)

# list stored by label 
index_label = [[] for i in range(c_num)]
for i in range(len(indeces_labels[1])):
    index_label[indeces_labels[1][i]].append(indeces_labels[0][i])
# print(index_label)
# print(len(index_label))

classes indx  [0, 1, 2]
users_dict {0: array([], dtype=int64), 1: array([], dtype=int64), 2: array([], dtype=int64), 3: array([], dtype=int64), 4: array([], dtype=int64), 5: array([], dtype=int64), 6: array([], dtype=int64), 7: array([], dtype=int64), 8: array([], dtype=int64), 9: array([], dtype=int64)}


In [135]:
from itertools import combinations
client_classes = []
comb = []
for i in list(combinations(list(range(0,c_num)), 2)):
    print(i)
    comb.append(i)
print(comb)

# classes of client
for i in range(num_users):
    client_classes.append(comb[i%c_num])
client_classes = np.array(client_classes)
c = client_classes.flatten()
print(client_classes)

# count of labels
label_count = collections.Counter(c)
print(label_count)

(0, 1)
(0, 2)
(1, 2)
[(0, 1), (0, 2), (1, 2)]
[[0 1]
 [0 2]
 [1 2]
 [0 1]
 [0 2]
 [1 2]
 [0 1]
 [0 2]
 [1 2]
 [0 1]]
Counter({0: 7, 1: 7, 2: 6})


In [136]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [137]:
for i in range(len(label_count)):
    index_label[i] = split(index_label[i], label_count[i])
    index_label[i] = list(index_label[i])

In [138]:
# for i in index_label:
#     print(i)

[[249, 131, 95, 54, 297, 260, 113, 139, 290, 158, 129, 243, 5, 269, 166, 284, 135], [165, 148, 108, 258, 110, 72, 183, 45, 282, 88, 77, 168, 55, 127, 94, 195, 170], [76, 286, 270, 1, 102, 25, 11, 32, 215, 201, 163, 57, 140, 69, 169, 27, 117], [238, 90, 23, 256, 237, 235, 141, 10, 71, 197, 133, 190, 298, 136, 178, 206], [112, 289, 146, 111, 29, 292, 149, 261, 119, 208, 49, 252, 97, 172, 211, 86], [193, 191, 279, 262, 80, 120, 15, 179, 103, 263, 218, 8, 12, 281, 162, 224], [231, 198, 93, 247, 155, 67, 26, 157, 28, 288, 204, 104, 105, 2, 37, 9]]
[[194, 128, 152, 192, 240, 99, 221, 122, 205, 24, 219, 210, 177], [41, 100, 48, 92, 62, 241, 109, 161, 244, 274, 34, 98, 236], [126, 276, 36, 18, 287, 114, 251, 91, 107, 273, 229, 228, 145], [245, 278, 171, 143, 13, 182, 46, 159, 242, 181, 87, 200, 246], [220, 151, 130, 295, 254, 30, 250, 142, 285, 3, 121, 203], [132, 283, 78, 16, 101, 180, 14, 176, 51, 280, 81, 123], [291, 253, 0, 234, 209, 275, 296, 43, 4, 39, 70, 230]]
[[227, 134, 17, 216, 207,

In [139]:
temp = []
users_dict = dict.fromkeys(range(10), [])
print("users_dict ", users_dict)
for i in range(len(client_classes)):
    for j in range(len(client_classes[i])):
        cur_cls = client_classes[i][j]
        temp = index_label[cur_cls].pop()
#         print("temp ", temp)
        users_dict[i] = np.concatenate((users_dict[i], np.array(temp)), axis=0).astype(int)

for i in range(len(users_dict)):
    users_dict[i] = set(users_dict[i])

users_dict  {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}


In [140]:
print(users_dict)

{0: {0, 2, 4, 9, 275, 26, 155, 28, 157, 288, 291, 37, 39, 296, 43, 67, 198, 70, 204, 209, 93, 230, 231, 104, 105, 234, 247, 253}, 1: {262, 263, 8, 264, 266, 12, 15, 144, 279, 281, 156, 162, 293, 42, 179, 60, 191, 193, 75, 80, 83, 89, 218, 223, 224, 225, 248, 96, 103, 233, 118, 120}, 2: {132, 137, 14, 16, 19, 150, 280, 283, 167, 44, 174, 47, 176, 175, 51, 180, 184, 189, 63, 199, 73, 74, 78, 79, 81, 82, 101, 123}, 3: {130, 3, 261, 142, 146, 149, 151, 29, 30, 285, 289, 292, 295, 172, 49, 203, 208, 211, 86, 220, 97, 111, 112, 119, 121, 250, 252, 254}, 4: {256, 259, 133, 136, 10, 141, 20, 277, 23, 31, 35, 298, 173, 178, 50, 58, 187, 186, 188, 190, 61, 66, 197, 71, 206, 84, 85, 90, 235, 237, 238, 124}, 5: {257, 268, 13, 143, 271, 278, 154, 159, 33, 38, 171, 299, 46, 181, 182, 53, 185, 196, 200, 213, 214, 87, 106, 242, 115, 116, 245, 246, 255}, 6: {1, 11, 140, 270, 273, 18, 145, 276, 25, 27, 286, 287, 32, 163, 36, 169, 57, 69, 201, 76, 215, 91, 228, 229, 102, 107, 114, 117, 251, 126}, 7: {258

In [ ]:
indeces = [int(i) for i in indeces]
print(indeces)
for i in range(num_users):
    np.random.seed(i)
    temp = set(np.random.choice(classes_indx, 2, replace=False))
    print(temp)
    classes_indx = list(set(classes_indx) - temp)
    for t in temp:
        users_dict[i] = np.concatenate((users_dict[i], indeces[t*images:(t+1)*images]), axis=0)